In [ ]:
import warnings
from pgmpy.estimators import BayesianEstimator
from pgmpy.models import BayesianModel
import numpy as np
from base.utils import confusion_matrix, acc
from data import data
from sklearn.model_selection import KFold


class MyBayesianModel(BayesianModel):
    def __init__(self, model_name, ebunch=None):
        super(MyBayesianModel, self).__init__(ebunch)
        self.model_name = model_name


if __name__ == '__main__':
    warnings.filterwarnings(action="ignore")
    df = data.load_data()
    models = []
    # naive 1
    models.append(MyBayesianModel("Naive1", [('e', 'm'),
                                             ('e', 'p'),
                                             ('e', 'd'),
                                             ('e', 's'),
                                             ('e', 'b'),
                                             ('e', 'l')]
                                  ))
    # naive 2
    models.append(MyBayesianModel("Naive2", [('e', 'm'),
                                             ('e', 'd'),
                                             ('e', 's'),
                                             ('e', 'l')]
                                  ))
    # naive 3
    models.append(MyBayesianModel("Naive3", [('e', 'm'),
                                             ('e', 'p'),
                                             ('e', 'd'),
                                             ('e', 's'),
                                             ('e', 'l')]
                                  ))
    # BayesNet1
    models.append(MyBayesianModel("BayesNet1", [('b', 's'),
                                                ('s', 'e'),
                                                ('m', 'e'),
                                                ('l', 'e'),
                                                ('d', 'p'),
                                                ('p', 'b')]
                                  ))
    models.append(MyBayesianModel("BayesNet2", [('b', 'e'),
                                                ('s', 'e'),
                                                ('m', 'e'),
                                                ('l', 'e'),
                                                ('p', 'e'),
                                                ('d', 'p'),
                                                ('p', 'b'),
                                                ('s', 'b')]
                                  ))

    models.append(MyBayesianModel("BayesNet3", [('b', 'e'),
                                                ('s', 'l'),
                                                ('m', 'e'),
                                                ('l', 'e'),
                                                ('p', 'm'),
                                                ('d', 'p'),
                                                ('p', 'b'),
                                                ('p', 's')]
                                  ))

    report = ""
    for model in models:
        model.add_nodes_from(['b', 'm', 'd', 'p', 'l', 's', 'e'])
        accs = []
        kf = KFold(n_splits=10, shuffle=True, random_state=0)
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            model.fit(train, estimator=BayesianEstimator)
            # run queries
            y_true = test[['e']].values.ravel()
            y_pred = model.predict(test[['b', 'm', 'd', 'p', 'l', 's']]).values.ravel()
            accs.append(acc(confusion_matrix(y_true, y_pred)))

        report += "Accuracy(%s) -> %f\n" % (model.model_name, np.asarray(accs).mean())

    print report
